<a href="https://colab.research.google.com/github/Shibli-Nomani/Deep-Learning/blob/main/01_ANN_churn_model_ud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Artifical Neural Network**

###Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [2]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [3]:
tf.__version__

'2.8.2'

## **Part-01 Data Preprocessing**

### Importing Dataset

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [138]:
path='/content/gdrive/MyDrive/Deep Learning-U/Churn_Modelling.csv'
df=pd.read_csv(path)
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [140]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [194]:
#features 
X=df.copy() #as RowNumber, Customer Id and Surname don't have any impact
X=df.drop(['RowNumber', 'CustomerId', 'Surname','Exited'], axis=1)
#Target
y=df['Exited']
print(X.shape)
print(y.shape)


(10000, 10)
(10000,)


In [180]:
X.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [181]:
y.head(5)

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [182]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [183]:
X.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
dtype: object

###Encoding Categorical Data

**Use Label Encoding**

In [195]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X.Gender= le.fit_transform(X['Gender'])

In [196]:
X.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57


In [197]:
X['Gender'].value_counts()

1    5457
0    4543
Name: Gender, dtype: int64

**One Hot Encoding with dummies**

In [201]:
#as there is no relationship amongs the countries (in Geography col), we use hot encoding
columns=['Geography']
for col in columns:
  one_hot_en=pd.get_dummies(X['Geography'], drop_first=False) 
  X=pd.concat((X,one_hot_en), axis=1)
X=X.drop(columns, axis=1)
X.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [187]:
#Alternative way of OneHotencoding(get the result in a list not dataframe)
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import OneHotEncoder
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') #for index=1; Geography
#X = pd.DataFrame(ct.fit_transform(X))


### **Splitting the dataset Train Test**

In [202]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state =0)

### **Feature Scaling**

In [203]:
from operator import xor
#applying over all values of X_train X_test
#Standard Scaler
x1=X_train.columns
x2=X_test.columns
from sklearn.preprocessing import StandardScaler 
sc= StandardScaler()
X_train[x1] = sc.fit_transform(X_train[x1])
X_test[x2] = sc.fit_transform(X_test[x2])
print('X_train Columns:')
print(X_train.head(3))
print('X_test Columns:')
print(X_test.head(3))

X_train Columns:
      CreditScore    Gender       Age    Tenure   Balance  NumOfProducts  \
7389     0.169582 -1.091687 -0.464608  0.006661 -1.215717       0.809503   
9275    -2.304559  0.916013  0.301026 -1.377440 -0.006312      -0.921591   
2995    -1.191196 -1.091687 -0.943129 -1.031415  0.579935      -0.921591   

      HasCrCard  IsActiveMember  EstimatedSalary    France   Germany     Spain  
7389   0.642595       -1.032270         1.106432 -1.014607 -0.569844  1.743090  
9275   0.642595        0.968738        -0.748664 -1.014607  1.754865 -0.573694  
2995   0.642595       -1.032270         1.485335  0.985604 -0.569844 -0.573694  
X_test Columns:
      CreditScore    Gender       Age    Tenure   Balance  NumOfProducts  \
9394    -0.561294 -1.113392 -0.394017  0.986971  0.859622      -0.873377   
898     -1.338478 -1.113392  0.076114 -1.084321  0.403122      -0.873377   
2398     0.583476 -1.113392  0.264167  0.986971  0.280118      -0.873377   

      HasCrCard  IsActiveMember  

##**Part-02 Building the ANN**

### **Initializing the ANN**

In [204]:
import keras
from keras.models import sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [205]:
ann=tf.keras.models.Sequential() #as Keras has already integrated in tensorflow. we don't need to import it individually

### Add Input and First Hidden Layer 

In [206]:
#whatever state we are it's reffered to use Dense Class
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #rectifired activation function

### Add Second Hidden Layer 

In [207]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Add Output Layer

In [208]:
#in output layer, we need to check o/p dimesion, here it is binary(0/1); we can say input nueron is 1
#activation is Sigmoid only for output binary. It's not noly providing the prediction outcome also probability of output
# for non-binary class, softmax -- activation
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


## **Part-03 Training ANN**

###**Compile the ANN**

In [209]:
#best optimizer is SGD. and adam is for SGD. It will adjust weight to reduce loss between predicted and actual
#for binary outcome, we must use loss function. for non-binary, categoriacl_crossentropy
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #can be chosen list of several metrics at a time for evaluation

###**Train the ANN on the whole training set**

In [210]:
#batch learning is more efficeint and more performent in ANN. Number of prediction we need to compare with same no of real result
#classic value of batch size is 32 to avoid too much time in hyperparameter use default value 32
#neural has train with certain no of epochs to improve the accuracy over time; it requires to get ultimate best predictions
ann.fit(X_train, y_train, batch_size=32, epochs=100)  #50 epochs are good enough for train it

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5068 - accuracy: 0.7857
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.7959
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4359 - accuracy: 0.7976
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4183 - accuracy: 0.8126
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4105 - accuracy: 0.8177
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4031 - accuracy: 0.8206
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3963 - accuracy: 0.8261
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3909 - accuracy: 0.8261
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3860 - accura

## **Part-04 Making Prediction and Model Evaluation**

###**Predict the result for single observation**

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

**Verify Table Sequency after OneHotEncoding**

In [213]:
X.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0


In [214]:
X.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'France', 'Germany',
       'Spain'],
      dtype='object')

**Compare object name df and labelencoding of categorical value**

In [215]:
df.iloc[[1,2]]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [216]:
X.iloc[[1,2]]

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0


**Predicted output**

In [221]:
#any input of predict will be 2d array [[]] wheater it is test, ensemble outcome/observation and any single value
#here geography is 'France', we need to enter the dummy variable as per encoding
#need to maintain the columns features as per X/X_train
#as we have done scaling in X_train and X_test, we also need to scale here. we only need to use transform method for predict / production stage
print('Predicted Value of Leave the Bank:', ann.predict(sc.transform([[600, 1, 40, 3, 60000, 2, 1, 1, 50000, 1, 0, 0]])))
print('Predicted Value of Leave the Bank:', ann.predict(sc.transform([[600, 1, 40, 3, 60000, 2, 1, 1, 50000, 1, 0, 0]]))>0.5)


Predicted Value of Leave the Bank: [[0.0203118]]
Predicted Value of Leave the Bank: [[False]]


As per prediction, customer will not leave the bank.

### **Predected value of X_test**

In [272]:

y_pred=ann.predict(X_test)
y_pred=np.where(y_pred <= 0.5, 0, 1)
y_pred=pd.DataFrame(y_pred)
y_pred.columns=['predicted']
y_pred

,predicted
0,0
1,0
2,0
3,0
4,0
...,...
1995,0
1996,0
1997,0
1998,0


###**Prepare Compare Table y_pred and y_test**

In [273]:
y_pred.merge(y_test, left_index=True, right_index=True)

,predicted,Exited
9,1,0
15,1,0
18,0,0
23,0,0
31,0,0
...,...,...
1984,0,0
1985,0,1
1986,0,0
1994,0,1


###**Making Confusion Matrix**

In [275]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1523   72]
 [ 214  191]]


0.857